**Note:** Run this notebook in classic SageMaker notebooks. Do NOT use SageMaker Studio notebook. You can also execute the commands locally from your desktop terminal if it is setup with AWS credentials.

Use `conda_python3` kernel if using SageMaker classic notebooks.

### Build a docker image locally

In [8]:
!docker build -t pyodbc-sage -f Dockerfile .

Sending build context to Docker daemon  95.23kB
Step 1/5 : FROM python:3.7-slim
 ---> b5900b90787e
Step 2/5 : RUN apt-get update  && apt-get install unixodbc -y  && apt-get install unixodbc-dev -y  && apt-get install freetds-dev -y  && apt-get install freetds-bin -y  && apt-get install tdsodbc -y  && apt-get install --reinstall build-essential -y
 ---> Using cache
 ---> 1c880f72312b
Step 3/5 : RUN echo "[FreeTDS]\nDescription = FreeTDS unixODBC Driver\nDriver = /usr/lib/x86_64-linux-gnu/odbc/libtdsodbc.so\nSetup = /usr/lib/x86_64-linux-gnu/odbc/libtdsS.so" >> /etc/odbcinst.ini
 ---> Using cache
 ---> 5e8819a57dc6
Step 4/5 : COPY requirements.txt ./
 ---> Using cache
 ---> cf82ca425832
Step 5/5 : RUN pip install --no-cache-dir -r requirements.txt
 ---> Using cache
 ---> 28bea596ed86
Successfully built 28bea596ed86
Successfully tagged pyodbc-sage:latest


### Push local docker image to AWS Elastic Container Registry (ECR)

In [9]:
%%sh

# Specify a name to your custom container
container_name=pyodbc-sage # make sure this name is same as the image name above
echo "Container Name: " ${container_name}

# Retrieve AWS account ID
account=$(aws sts get-caller-identity --query Account --output text)

# Get the AWS region defined in the current configuration (default to us-east-1 if not defined)
region=$(aws configure get region)
region=${region:-us-east-1}

echo "Account: " ${account}
echo "Region: "${region}

repository="${account}.dkr.ecr.${region}.amazonaws.com"
echo "ECR Repository: " ${repository}

image="${account}.dkr.ecr.${region}.amazonaws.com/${container_name}:latest"
echo "ECR Image URI: " ${image}

# If the ECR repository does not exist, create it.
aws ecr describe-repositories --repository-names ${container_name} > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name ${container_name} > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin ${repository}

# Tag the local image with ECR image name
docker tag ${container_name} ${image}

# Finally, push the local docker image to ECR with the full ECR image name
docker push ${image}

Container Name:  pyodbc-sage
Account:  892313895307
Region: us-east-1
ECR Repository:  892313895307.dkr.ecr.us-east-1.amazonaws.com
ECR Image URI:  892313895307.dkr.ecr.us-east-1.amazonaws.com/pyodbc-sage:latest
Login Succeeded
The push refers to repository [892313895307.dkr.ecr.us-east-1.amazonaws.com/pyodbc-sage]
e07692b3f4a8: Preparing
25ad686732a2: Preparing
8ac03f2907c0: Preparing
d57d30b7f01f: Preparing
e8f55de3cd48: Preparing
f70d531659d0: Preparing
a2bef98ac977: Preparing
53264a515eac: Preparing
814bff734324: Preparing
f70d531659d0: Waiting
a2bef98ac977: Waiting
53264a515eac: Waiting
814bff734324: Waiting
25ad686732a2: Pushed
8ac03f2907c0: Pushed
f70d531659d0: Pushed
e8f55de3cd48: Pushed
53264a515eac: Pushed
a2bef98ac977: Pushed
814bff734324: Pushed
e07692b3f4a8: Pushed
d57d30b7f01f: Pushed
latest: digest: sha256:6e12056f706e03d1da0a6f662cc5faa1a83fae56d0558fb63a280eafaea70844 size: 2208


WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

